# Simulation 4 - ATLAS

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV, KFold

import torch
from torch.utils.data import Dataset,DataLoader
from torch import optim
import torch.nn as nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

## Random Feature

In [68]:
def sample_1d(pdf, gamma):
    if pdf=='G':
        w=torch.randn(1)*gamma
        return w
    elif pdf=='L':
        w=torch.distributions.laplace.Laplace(torch.tensor([0.0]), torch.tensor([1.0])).sample()*gamma
        return w
    elif pdf=='C':
        w=torch.distributions.cauchy.Cauchy(torch.tensor([0.0]), torch.tensor([1.0])).sample()*gamma
        return w
    
def sample(pdf, gamma, d):
    return torch.tensor([sample_1d(pdf, gamma) for _ in range(d)])

class RandomFourierFeature:
    """Random Fourier Feature
    Parameters
    ----------
    d : int
        Input space dimension
    D : int
        Feature space dimension
    W : shape (D,d)
    b : shape (D)
    kernel : char
        Kernel to use; 'G', 'L', or 'C'
    gamma : float
        pdf parameter
    """

    def __init__(self, d, D, W=None, b=None, kernel='G', gamma=1):

        self.d = d
        self.D = D
        self.gamma = gamma

        kernel = kernel.upper()
        if kernel not in ['G', 'L', 'C']:
            raise Exception('Invalid Kernel')
        self.kernel = kernel

        if W is None or b is None:
            self.create()
        else:
            self.__load(W, b)

    def __load(self, W, b):
        """Load from existing Arrays"""

        self.W = W.reshape([self.D, self.d])
        self.b = b
    

    def create(self):
        """Create a d->D fourier random feature"""

        self.b = torch.rand(self.D)*2*torch.pi
        self.W = sample(self.kernel, self.gamma, self.d*self.D).reshape(self.D,self.d)

    def transform(self, x):
        """Transform a vector using this feature
        Parameters
        ----------
        x : (shape=(n,d))
            to transform; must be single dimension vector
        Returns
        -------
        x : (shape=(n,D))
            Feature space transformation of x
        """
       
        result=torch.sqrt(torch.tensor([2.0/self.D])) * torch.cos( self.W @ x.T  + (self.b.reshape(-1,1) @ torch.ones(len(x)).reshape(1,-1))) 
        return result.T

data generation

In [69]:
n=2000
p=64
np.random.seed(0)
data=np.random.uniform(0,1, (n,p)) #n points
np.random.seed(1)
noise=np.random.normal(0,1,n)

#function
def g1(x):
    return -2*np.sin(2*np.pi*x)
def g2(x):
    return x**2-1/3
def g3(x):
    return x-1/2
def g4(x):
    return np.exp(x)+np.exp(1)-1
a1=1
a4=4
def a2(x):
    return 2/np.sqrt(2*np.pi)*np.exp(-(x-1)**2/2)
def a3(x):
    return 3*np.cos(2*np.pi*x)

y=a1*g1(data[:,0])+a2(data[:,0])*g2(data[:,1])+a3(data[:,0])*g3(data[:,2])+a4*g4(data[:,3])+noise


np.random.seed(2)
calibration_x=np.random.uniform(0,1, (n,p)) #n points
np.random.seed(3)
calibration_noise=np.random.normal(0,1,n)
calibration_y=a1*g1(calibration_x[:,0])+a2(calibration_x[:,0])*g2(calibration_x[:,1])+a3(calibration_x[:,0])*g3(calibration_x[:,2])+a4*g4(calibration_x[:,3])+calibration_noise



np.random.seed(4)
test_x=np.random.uniform(0,1, (2*n,p)) #n points
np.random.seed(5)
test_noise=np.random.normal(0,1,2*n)
test_y=a1*g1(test_x[:,0])+a2(test_x[:,0])*g2(test_x[:,1])+a3(test_x[:,0])*g3(test_x[:,2])+a4*g4(test_x[:,3])+test_noise

In [70]:
train_x=data
train_y=y

total_x=np.vstack((train_x,calibration_x))
total_y=np.hstack((train_y,calibration_y))

nntrain_x = torch.from_numpy(train_x).float()
nntrain_y = torch.squeeze(torch.from_numpy(train_y).float()) 
nntest_x= torch.from_numpy(test_x).float()
nntest_y = torch.squeeze(torch.from_numpy(test_y).float())

class mydataset(Dataset):
    def __init__(self, x, y):
        self._x = x
        self._y = y
        self._len = len(x)

    def __getitem__(self, item): 
        return self._x[item], self._y[item]

    def __len__(self):
        return self._len

## KRR

In [71]:
t0=time.time()
krr = KernelRidge(kernel='rbf',gamma=1/2)
param_grid = {
    'alpha': [1e-7,1e-6,1e-5,1e-4,1e-3]
}

kf = KFold(n_splits=5, shuffle=True)  
grid_search = GridSearchCV(krr, param_grid, cv=kf)
grid_search.fit(total_x, total_y)

print("Best parameters found: ", grid_search.best_params_)

best_krr = grid_search.best_estimator_
krr_pred = best_krr.predict(test_x)

t1=time.time()-t0
print(mean_squared_error(test_y,krr_pred))
print("Time:",t1)

Best parameters found:  {'alpha': 1e-07}
15.554395988711656
Time: 16.291438102722168


## RF

In [72]:
t0=time.time()
model=Ridge()
param_grid = {
    'alpha': [1e-4,1e-3,1e-2,1e-1,1]
}
rff=RandomFourierFeature(p,500,kernel='G',gamma=1)
total_feature=rff.transform(total_x)
test_feature=rff.transform(test_x)

kf = KFold(n_splits=5, shuffle=True)  
grid_search = GridSearchCV(model, param_grid, cv=kf)
grid_search.fit(total_feature, total_y)

print("Best parameters found: ", grid_search.best_params_)

best_model = grid_search.best_estimator_
rf_pred = best_model.predict(test_feature)

t1=time.time()-t0
print(mean_squared_error(test_y,rf_pred))
print("Time:",t1)

Best parameters found:  {'alpha': 1}
6.975625297775046
Time: 1.5110466480255127


# MLKM

In [73]:
rff1=RandomFourierFeature(p,256,kernel='G',gamma=1)
rff2=RandomFourierFeature(16,16,kernel='G',gamma=1)

class KernelNet(nn.Module): 
    def __init__(self):
        super(KernelNet, self).__init__()
        self.fc1 = nn.Linear(256,16)
        self.fc2 = nn.Linear(16, 1)
        
    def forward(self, x):
        x = rff1.transform(x)
        x=self.fc1(x)
        x = rff2.transform(x)
        return self.fc2(x)

#initialize
def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.normal_(m.weight,mean=0,std=0.5)
    if type(m) == nn.Linear:
        torch.nn.init.uniform_(m.weight,a=-0.1,b=0.1)
        m.bias.data.fill_(0.01)


def MultiLayerKfold(kfold,param_grid):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    param_error=[]
    for param in tqdm(param_grid):
        k_error=[]
        for k in range(kfold):
            Kfold_val_x, Kfold_val_y=nntrain_x[int(n/5)*k:int(n/5)*(k+1)], nntrain_y[int(n/5)*k:int(n/5)*(k+1)]
            Kfold_train_x = torch.cat((nntrain_x[:int(n/5)*k],nntrain_x[int(n/5)*(k+1):]),dim = 0)
            Kfold_train_y = torch.cat((nntrain_y[:int(n/5)*k],nntrain_y[int(n/5)*(k+1):]),dim = 0)
           
            net = KernelNet()  #### KernelNet!
            net = net.to(device)
            torch.manual_seed(1)
            net.apply(init_weights)
            criterion=nn.MSELoss() 
            optimizer=optim.SGD(net.parameters(),lr=1e-3,momentum=0.9,weight_decay=param) #optim.Adam(...)
            kfoldtrain_loader = DataLoader(mydataset(Kfold_train_x, Kfold_train_y),batch_size=128, shuffle=True)
            
            for epoch in range(2000): 
                for x, y in kfoldtrain_loader: #for batch, (x, y) in enumerate(train_loader): 
                    x, y = x.to(device), y.to(device)
                    # Compute prediction error
                    y_pred = net(x)
                    y_pred = torch.squeeze(y_pred)
                    train_loss = criterion(y_pred, y)
                    # Backpropagation
                    optimizer.zero_grad() 
                    train_loss.backward()
                    optimizer.step()
            
            x0=Kfold_val_x[:].float()
            with torch.no_grad():
                x0 = x0.to(device)
                val_pred = net(x0)
            
            k_error.append(mean_squared_error(val_pred,Kfold_val_y))
        
        param_error.append(np.mean(k_error))
        
    print(param_error) 
    return param_grid[np.argmin(param_error)]  

bestweight=MultiLayerKfold(5,[1e-2,1e-1,1])     
print(bestweight)

100%|██████████| 3/3 [12:24<00:00, 248.18s/it]

[13.766411, 7.976598, 11.482803]
0.1


In [74]:
train_loader = DataLoader(mydataset(nntrain_x, nntrain_y),batch_size=128, shuffle=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = KernelNet()
net = net.to(device)
torch.manual_seed(1)
net.apply(init_weights)
print(net)
criterion=nn.MSELoss() 
optimizer=optim.SGD(net.parameters(),lr=1e-3,momentum=0.9,weight_decay=0.5) #optim.Adam(...)

loss=[]
kernelnn_trainloss=[]
kernelnn_testloss=[]
t0 = time.time()
for epoch in range(2000): 
    for x, y in train_loader: #for batch, (x, y) in enumerate(train_loader): 
        x, y = x.to(device), y.to(device)
        # Compute prediction error
        y_pred = net(x)
        y_pred = torch.squeeze(y_pred)
        train_loss = criterion(y_pred, y)
        loss.append(train_loss)
        # Backpropagation
        optimizer.zero_grad() 
        train_loss.backward()
        optimizer.step()
    
    x0=torch.from_numpy(train_x[:]).float()
    with torch.no_grad():
        x0 = x0.to(device)
        train_pred = net(x0)
    
    x0=torch.from_numpy(test_x[:]).float()
    with torch.no_grad():
        x0 = x0.to(device)
        test_pred = net(x0)
    
    kernelnn_trainloss.append(mean_squared_error(train_pred,train_y))
    kernelnn_testloss.append(mean_squared_error(test_pred,test_y))
    
    if epoch % 100 == 0:        
        print(f'''epoch {epoch}
            Train set - loss: {kernelnn_trainloss[-1]}
            Test set - loss: {kernelnn_testloss[-1]}
            ''')
        
    
dnn_fit = time.time() - t0
print("KernelNet complexity and model fitted in %.3f s" % dnn_fit)

KernelNet(
  (fc1): Linear(in_features=256, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
)
epoch 0
            Train set - loss: 96.63143471362186
            Test set - loss: 96.15927819672774
            
epoch 100
            Train set - loss: 8.516716571715504
            Test set - loss: 8.603429184808228
            
epoch 200
            Train set - loss: 8.45170839624862
            Test set - loss: 8.540438385134864
            
epoch 300
            Train set - loss: 8.517860171257835
            Test set - loss: 8.604517692395312
            
epoch 400
            Train set - loss: 8.471129423432684
            Test set - loss: 8.55916924977642
            
epoch 500
            Train set - loss: 8.496592375383292
            Test set - loss: 8.583876525073812
            
epoch 600
            Train set - loss: 8.482962270358437
            Test set - loss: 8.57069639474534
            
epoch 700
            Train set - loss: 8.4530

In [75]:
kernel_x0=torch.from_numpy(test_x[:]).float()
with torch.no_grad():
    kernel_x0 = kernel_x0.to(device)
    kernel_pred = net(kernel_x0)

### Conformal confidence bands for MLKM

In [76]:
##conformal prediction comparison
#predict
x0=torch.from_numpy(calibration_x[:]).float()
with torch.no_grad():
    x0 = x0.to(device)
    pred = net(x0)
    score=np.abs(pred.reshape(-1)-calibration_y[:])
sorted_score, sorted_indices=torch.sort(score)
q=(len(calibration_x)+1)*0.95
print(np.ceil(q))
a=sorted_score[int(np.ceil(q))]

coverage=0
x0=torch.from_numpy(test_x[:]).float()
with torch.no_grad():
    x0 = x0.to(device)
    pred = net(x0)
for i in range(len(test_x)):
    if pred.detach().numpy()[i][0]-a<test_y[i] and pred.detach().numpy()[i][0]+a>test_y[i]:
        coverage=coverage+1
coverage=coverage/len(test_x)

print("length",2*a)
print("95 coverage",coverage)

1901.0
length tensor(11.3901, dtype=torch.float64)
95 coverage 0.9465


## RK

In [77]:
train_loader = DataLoader(mydataset(nntrain_x, nntrain_y),batch_size=128, shuffle=True)

rff0=RandomFourierFeature(p,256,kernel='G',gamma=1)
rff1=RandomFourierFeature(16,16,kernel='G',gamma=1)

class ResidualBlock(nn.Module):
    def __init__(self,infeatures,outfeatures,rff):
        super(ResidualBlock,self).__init__()
        self.infeatures = infeatures
        self.outfeatures = outfeatures
        self.rff=rff
        
        self.fc1 = nn.Linear(infeatures,outfeatures)
        self.fc2 = nn.Linear(outfeatures,outfeatures)
    
    def forward(self, x):
        rff=self.rff
        x = self.fc1(x)
        y = rff.transform(x)
        y = self.fc2(y)
        return x+y

class ResKernelNet(nn.Module): 
    def __init__(self):
        super(ResKernelNet, self).__init__()
        self.rblock1 = ResidualBlock(256,16,rff1)
        self.fc2 =nn.Linear(16,1)
 
    def forward(self, x):
        x = rff0.transform(x)
        x = self.rblock1(x)
        return self.fc2(x)

#initialize
def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.normal_(m.weight,mean=0,std=0.5)
    if type(m) == nn.Linear:
        torch.nn.init.uniform_(m.weight,a=-0.1,b=0.1)
        m.bias.data.fill_(0.01)

def MultiLayerKfold(kfold,param_grid):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    param_error=[]
    for param in tqdm(param_grid):
        k_error=[]
        for k in range(kfold):
            Kfold_val_x, Kfold_val_y=nntrain_x[int(n/5)*k:int(n/5)*(k+1)], nntrain_y[int(n/5)*k:int(n/5)*(k+1)]
            Kfold_train_x = torch.cat((nntrain_x[:int(n/5)*k],nntrain_x[int(n/5)*(k+1):]),dim = 0)
            Kfold_train_y = torch.cat((nntrain_y[:int(n/5)*k],nntrain_y[int(n/5)*(k+1):]),dim = 0)
           
            net = ResKernelNet()  #### ResKernelNet!
            net = net.to(device)
            torch.manual_seed(1)
            net.apply(init_weights)
            criterion=nn.MSELoss() 
            optimizer=optim.SGD(net.parameters(),lr=1e-3,momentum=0.9,weight_decay=param) #optim.Adam(...)
            kfoldtrain_loader = DataLoader(mydataset(Kfold_train_x, Kfold_train_y),batch_size=128, shuffle=True)
            
            for epoch in range(2000): 
                for x, y in kfoldtrain_loader: #for batch, (x, y) in enumerate(train_loader): 
                    x, y = x.to(device), y.to(device)
                    # Compute prediction error
                    y_pred = net(x)
                    y_pred = torch.squeeze(y_pred)
                    train_loss = criterion(y_pred, y)
                    # Backpropagation
                    optimizer.zero_grad() 
                    train_loss.backward()
                    optimizer.step()
            
            x0=Kfold_val_x[:].float()
            with torch.no_grad():
                x0 = x0.to(device)
                val_pred = net(x0)
            
            k_error.append(mean_squared_error(val_pred,Kfold_val_y))
        
        param_error.append(np.mean(k_error))
        
    print(param_error) 
    return param_grid[np.argmin(param_error)]  

bestweight=MultiLayerKfold(5,[1e-2,1e-1,1])     
print(bestweight)

100%|██████████| 3/3 [14:07<00:00, 282.54s/it]

[18.084696, 11.777431, 7.09048]
1


In [78]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = ResKernelNet()
net = net.to(device)
torch.manual_seed(1)
net.apply(init_weights)
print(net)
criterion=nn.MSELoss() 
optimizer=optim.SGD(net.parameters(),lr=1e-3,momentum=0.9,weight_decay=bestweight) #optim.Adam(...)

loss=[]
reskernel_trainloss=[]
reskernel_testloss=[]
t0 = time.time()
for epoch in range(2000): 
    for x, y in train_loader: #for batch, (x, y) in enumerate(train_loader): 
        x, y = x.to(device), y.to(device)
        # Compute prediction error
        y_pred = net(x)
        y_pred = torch.squeeze(y_pred)
        train_loss = criterion(y_pred, y)
        loss.append(train_loss)
        # Backpropagation
        optimizer.zero_grad() 
        train_loss.backward()
        optimizer.step()
    
    x0=torch.from_numpy(train_x[:]).float()
    with torch.no_grad():
        x0 = x0.to(device)
        train_pred = net(x0)
    
    x0=torch.from_numpy(test_x[:]).float()
    with torch.no_grad():
        x0 = x0.to(device)
        test_pred = net(x0)
    
    reskernel_trainloss.append(mean_squared_error(train_pred,train_y))
    reskernel_testloss.append(mean_squared_error(test_pred,test_y))
    
    if epoch % 100 == 0:        
        print(f'''epoch {epoch}
            Train set - loss: {reskernel_trainloss[-1]}
            Test set - loss: {reskernel_testloss[-1]}
            ''')
        
    
dnn_fit = time.time() - t0
print("Residual KernelNet complexity and model fitted in %.3f s" % dnn_fit)

ResKernelNet(
  (rblock1): ResidualBlock(
    (fc1): Linear(in_features=256, out_features=16, bias=True)
    (fc2): Linear(in_features=16, out_features=16, bias=True)
  )
  (fc2): Linear(in_features=16, out_features=1, bias=True)
)
epoch 0
            Train set - loss: 86.57530268921099
            Test set - loss: 86.14614601422588
            
epoch 100
            Train set - loss: 6.7771559237301915
            Test set - loss: 7.251644321119962
            
epoch 200
            Train set - loss: 6.821556617952419
            Test set - loss: 7.293870149105674
            
epoch 300
            Train set - loss: 6.785458617454458
            Test set - loss: 7.265033239725731
            
epoch 400
            Train set - loss: 6.8807355864958915
            Test set - loss: 7.35087929635438
            
epoch 500
            Train set - loss: 6.796436033622054
            Test set - loss: 7.26574479407241
            
epoch 600
            Train set - loss: 6.803331817579415
    

In [79]:
rk_x0=torch.from_numpy(test_x[:]).float()
with torch.no_grad():
    rk_x0 = rk_x0.to(device)
    rk_pred = net(rk_x0)

In [80]:
##conformal prediction comparison
#predict
x0=torch.from_numpy(calibration_x[:]).float()
with torch.no_grad():
    x0 = x0.to(device)
    pred = net(x0)
    score=np.abs(pred.reshape(-1)-calibration_y[:])
sorted_score, sorted_indices=torch.sort(score)
q=(len(calibration_x)+1)*0.95
print(np.ceil(q))
a=sorted_score[int(np.ceil(q))]

coverage=0
x0=torch.from_numpy(test_x[:]).float()
with torch.no_grad():
    x0 = x0.to(device)
    pred = net(x0)
for i in range(len(test_x)):
    if pred.detach().numpy()[i][0]-a<test_y[i] and pred.detach().numpy()[i][0]+a>test_y[i]:
        coverage=coverage+1
coverage=coverage/len(test_x)

print("length",2*a)
print("95 coverage",coverage)

1901.0
length tensor(10.3862, dtype=torch.float64)
95 coverage 0.95125


In [81]:
print(mean_squared_error(krr_pred,test_y))
print(mean_squared_error(rf_pred,test_y))
print(mean_squared_error(kernel_pred,test_y))
print(mean_squared_error(rk_pred,test_y))

15.554395988711656
6.975625297775046
8.566347232556026
7.309845424453881
